# Dislocations

This Notebook follows the workflow for generating dislocation-containing systems and analyzing the resulting structure using different techniques.

__Global workflow order:__ 

This Notebook requires

- The "unique_crystals.csv" file generated by the "3. Crystal relaxation" Notebook.

- calculation_elastic_constants_static records generated by the "4. Static elastic constants" Notebook.

**Library imports**

In [1]:
# Standard Python libraries
from __future__ import (absolute_import, print_function,
                        division, unicode_literals)

# http://www.numpy.org/
import numpy as np

from IPython.core.display import display, HTML

# https://pandas.pydata.org/
import pandas as pd

from DataModelDict import DataModelDict as DM

# https://github.com/usnistgov/atomman
import atomman.unitconvert as uc

# https://github.com/usnistgov/iprPy
import iprPy
print('iprPy version', iprPy.__version__)

iprPy version 0.8.3


## 0. Access database

### Load database

In [2]:
database = iprPy.load_database('demo')

### Load unique crystals

In [3]:
unique_crystals = pd.read_csv('unique_crystals.csv')
print(len(unique_crystals))

104


### Specify prototype-composition delimiters

In [4]:
proto_comps = {}
proto_comps['A1--Cu--fcc'] = ['Ne', 'Al', 'AlS', 'Ar', 'Ca', 'Fe', 'FeS', 'Ni', 'Cu', 'CuS', 
                              'Kr', 'Sr', 'Rh', 'Pd', 'Ag', 'Xe', 'Ir', 'Pt', 'Au', 'Pb', 
                              'Ac', 'Ce', 'Yb', 'Th']
proto_comps['A2--W--bcc'] = ['Li', 'Na', 'K', 'V', 'Cr', 'Fe', 'FeS', 'Rb', 'Nb', 'Mo', 
                             'Cs', 'Ba', 'Ta', 'W', 'Eu']
proto_comps['A3--Mg--hcp'] = ['H', 'He', 'Be', 'Mg', 'MgS', 'Sc', 'Ti', 'Co', 'Zn', 'Y', 
                              'Zr', 'Tc', 'Ru', 'Cd', 'Hf', 'Re', 'Os' 'Tl', 'Gd', 'Tb', 
                              'Dy', 'Ho', 'Er', 'Tm', 'Lu']
proto_comps['A4--C--dc'] = ['C', 'Si', 'SiS', 'Ge', 'Sn']

### Limit unique crystals using prototype-composition delimiters

In [5]:
proto_parents = {}
unique_crystal_keys = []
for prototype in proto_comps:
    proto_parents[prototype] = unique_crystals[(unique_crystals.prototype == prototype)
                                              &(unique_crystals.composition.isin(proto_comps[prototype]))].calc_key.tolist()
    print(len(proto_parents[prototype]), prototype)
    unique_crystal_keys.extend(proto_parents[prototype])
print(len(unique_crystal_keys), 'total')

3 A1--Cu--fcc
1 A2--W--bcc
0 A3--Mg--hcp
0 A4--C--dc
4 total


## 1. dislocation_monopole calculation

The dislocation monopole configuration is best for non-planar dislocations, like the bcc screw dislocation.

In [6]:
calculation = iprPy.load_calculation('dislocation_monopole')
run_directory = iprPy.load_run_directory('demo_4')

### Build input_dict

In [7]:
print(calculation.allkeys)

['lammps_command', 'mpi_command', 'length_unit', 'pressure_unit', 'energy_unit', 'force_unit', 'dislocation_boundarywidth', 'dislocation_boundaryshape', 'potential_file', 'potential_content', 'potential_dir', 'load_file', 'load_content', 'load_style', 'family', 'load_options', 'symbols', 'box_parameters', 'elasticconstants_file', 'elasticconstants_content', 'C11', 'C12', 'C13', 'C14', 'C15', 'C16', 'C22', 'C23', 'C24', 'C25', 'C26', 'C33', 'C34', 'C35', 'C36', 'C44', 'C45', 'C46', 'C55', 'C56', 'C66', 'C_M', 'C_lambda', 'C_mu', 'C_E', 'C_nu', 'C_K', 'sizemults', 'dislocation_file', 'dislocation_content', 'dislocation_family', 'dislocation_burgersvector', 'dislocation_stroh_m', 'dislocation_stroh_n', 'dislocation_lineboxvector', 'a_uvw', 'b_uvw', 'c_uvw', 'atomshift', 'randomseed', 'annealtemperature', 'annealsteps', 'energytolerance', 'forcetolerance', 'maxiterations', 'maxevaluations', 'maxatommotion']


In [8]:
input_script = """
# Commands and executables
lammps_command              lmp_mpi
#mpi_command                 c:\\Program Files\\MPICH2\\bin\\mpiexec -localonly 4

# Build load information from crystal_space_group results
buildcombos                 elasticparent load_file parent
parent_record               calculation_elastic_constants_static
parent_load_key             system-info
parent_strainrange          1e-7

# Build defect records
buildcombos                 defect dislocation_file
defect_record               dislocation_monopole
defect_id                   A2--W--bcc--a-2-111--0-screw--{110}

# System manipulations
a_uvw                      
b_uvw                      
c_uvw                 
atomshift                   
sizemults                   0 2 -24 24 -40 40

# Units that input/output values are in
length_unit                 
pressure_unit               
energy_unit                 
force_unit                  

# Run parameters
dislocation_boundarywidth   
dislocation_boundaryshape
annealtemperature           10
annealsteps                 100
randomseed
energytolerance             
forcetolerance              
maxiterations               
maxevaluations              
maxatommotion               
"""

with open('input_script.in', 'w') as f:
    f.write(input_script)

### Parse input script

In [9]:
with open('input_script.in') as f:
    input_dict = iprPy.input.parse(f, singularkeys=calculation.singularkeys)

### Add parent_load_file list using unique crystals

In [10]:
input_dict['parent_load_file'] = []
for key in unique_crystal_keys:
    input_dict['parent_load_file'].append(key + '.json')

In [11]:
input_dict

{'lammps_command': 'lmp_mpi',
 'buildcombos': ['elasticparent load_file parent', 'defect dislocation_file'],
 'parent_record': 'calculation_elastic_constants_static',
 'parent_load_key': 'system-info',
 'parent_strainrange': '1e-7',
 'defect_record': 'dislocation_monopole',
 'defect_id': 'A2--W--bcc--a-2-111--0-screw--{110}',
 'sizemults': '0 2 -24 24 -40 40',
 'annealtemperature': '10',
 'annealsteps': '100',
 'parent_load_file': ['5028ecce-272a-4a11-a3e4-10f97993fb8e.json',
  '3fe58722-8e95-4489-860e-e8ac58261306.json',
  'a11f2008-6ab6-4f9b-b434-2d1dcfa935f7.json',
  '7ae00795-a00a-49f7-9511-a2dd795a3f77.json']}

### Prepare calculations

In [12]:
database.prepare(run_directory, calculation, **input_dict)

In [13]:
database.check_records(calculation.record_style)

In database style local at C:\Users\lmh1\Documents\calculations\ipr\demo :
- 1 of style calculation_dislocation_monopole
 - 0 are complete
 - 1 still to run
 - 0 issued errors


### Run calculations

In [14]:
database.runner(run_directory)

Runner started with pid 14408
No simulations left to run


In [15]:
results_df = database.get_records_df(style=calculation.record_style)
error_df = results_df[results_df.status=='error']
print(len(error_df), 'calculations issued errors:')
errors = []
for error in error_df.error:
    lines = error.splitlines()
    err = ''
    for i in range(len(lines)-1, -1, -1):
        if 'Error:' in lines[i]:
            err = '\n'.join(lines[i:-1])
            break
        if i == 0:
            err = error
    errors.append(err)
for error in np.unique(errors):
    print(error)

0 calculations issued errors:


## 2. dislocation_periodic_array calculation

The periodic array of dislocations configuration is best suited for planar dislocations with large widths.

In [16]:
calculation = iprPy.load_calculation('dislocation_periodic_array')
run_directory = iprPy.load_run_directory('demo_4')

### Build input_dict

In [17]:
print(calculation.allkeys)

['lammps_command', 'mpi_command', 'length_unit', 'pressure_unit', 'energy_unit', 'force_unit', 'duplicatecutoff', 'boundarywidth', 'onlyuselinear', 'potential_file', 'potential_content', 'potential_dir', 'load_file', 'load_content', 'load_style', 'family', 'load_options', 'symbols', 'box_parameters', 'elasticconstants_file', 'elasticconstants_content', 'C11', 'C12', 'C13', 'C14', 'C15', 'C16', 'C22', 'C23', 'C24', 'C25', 'C26', 'C33', 'C34', 'C35', 'C36', 'C44', 'C45', 'C46', 'C55', 'C56', 'C66', 'C_M', 'C_lambda', 'C_mu', 'C_E', 'C_nu', 'C_K', 'sizemults', 'dislocation_file', 'dislocation_content', 'dislocation_family', 'dislocation_burgersvector', 'dislocation_stroh_m', 'dislocation_stroh_n', 'dislocation_lineboxvector', 'a_uvw', 'b_uvw', 'c_uvw', 'atomshift', 'randomseed', 'annealtemperature', 'annealsteps', 'energytolerance', 'forcetolerance', 'maxiterations', 'maxevaluations', 'maxatommotion']


In [18]:
input_script = """
# Commands and executables
lammps_command              lmp_mpi
#mpi_command                 c:\\Program Files\\MPICH2\\bin\\mpiexec -localonly 4

# Build load information from crystal_space_group results
buildcombos                 elasticparent load_file parent
parent_record               calculation_elastic_constants_static
parent_load_key             system-info
parent_strainrange          1e-7

# Build defect records
buildcombos                 defect dislocation_file
defect_record               dislocation_monopole
defect_id                   A1--Cu--fcc--a-2-110--0-screw--{111}

# System manipulations
a_uvw                      
b_uvw                      
c_uvw                 
atomshift                   
sizemults                   0 1 -102 102 -22 22

# Units that input/output values are in
length_unit                 
pressure_unit               
energy_unit                 
force_unit                  

# Run parameters
annealtemperature           10
annealsteps                 100
randomseed
energytolerance             
forcetolerance              
maxiterations               
maxevaluations              
maxatommotion               
duplicatecutoff             
boundarywidth               
onlyuselinear               True
"""

with open('input_script.in', 'w') as f:
    f.write(input_script)

In [24]:
input_script = """
# Commands and executables
lammps_command              lmp_mpi
#mpi_command                 c:\\Program Files\\MPICH2\\bin\\mpiexec -localonly 4

# Build load information from crystal_space_group results
buildcombos                 elasticparent load_file parent
parent_record               calculation_elastic_constants_static
parent_load_key             system-info
parent_strainrange          1e-7

# Build defect records
buildcombos                 defect dislocation_file
defect_record               dislocation_monopole
defect_id                   A1--Cu--fcc--a-2-110--90-edge--{100}

# System manipulations
a_uvw                      
b_uvw                      
c_uvw                 
atomshift                   
sizemults                   0 1 -176 176 -38 38

# Units that input/output values are in
length_unit                 
pressure_unit               
energy_unit                 
force_unit                  

# Run parameters
annealtemperature           10
annealsteps                 100
randomseed
energytolerance             
forcetolerance              
maxiterations               
maxevaluations              
maxatommotion               
duplicatecutoff             
boundarywidth               
onlyuselinear               
"""

with open('input_script.in', 'w') as f:
    f.write(input_script)

In [29]:
input_script = """
# Commands and executables
lammps_command              lmp_mpi
#mpi_command                 c:\\Program Files\\MPICH2\\bin\\mpiexec -localonly 4

# Build load information from crystal_space_group results
buildcombos                 elasticparent load_file parent
parent_record               calculation_elastic_constants_static
parent_load_key             system-info
parent_strainrange          1e-7

# Build defect records
buildcombos                 defect dislocation_file
defect_record               dislocation_monopole
defect_id                   A1--Cu--fcc--a-2-110--90-edge--{111}

# System manipulations
a_uvw                      
b_uvw                      
c_uvw                 
atomshift                   
sizemults                   0 1 -176 176 -22 22

# Units that input/output values are in
length_unit                 
pressure_unit               
energy_unit                 
force_unit                  

# Run parameters
annealtemperature           10
annealsteps                 100
randomseed
energytolerance             
forcetolerance              
maxiterations               
maxevaluations              
maxatommotion               
duplicatecutoff             
boundarywidth               
onlyuselinear               
"""

with open('input_script.in', 'w') as f:
    f.write(input_script)

In [35]:
input_script = """
# Commands and executables
lammps_command              lmp_mpi
#mpi_command                 c:\\Program Files\\MPICH2\\bin\\mpiexec -localonly 4

# Build load information from crystal_space_group results
buildcombos                 elasticparent load_file parent
parent_record               calculation_elastic_constants_static
parent_load_key             system-info
parent_strainrange          1e-7

# Build defect records
buildcombos                 defect dislocation_file
defect_record               dislocation_monopole
defect_id                   A2--W--bcc--a-2-111--90-edge--{110}

# System manipulations
a_uvw                      
b_uvw                      
c_uvw                 
atomshift                   
sizemults                   0 1 -57 57 -26 26

# Units that input/output values are in
length_unit                 
pressure_unit               
energy_unit                 
force_unit                  

# Run parameters
annealtemperature           10
annealsteps                 100
randomseed
energytolerance             
forcetolerance              
maxiterations               
maxevaluations              
maxatommotion               
duplicatecutoff             
boundarywidth               
onlyuselinear               
"""

with open('input_script.in', 'w') as f:
    f.write(input_script)

In [41]:
input_script = """
# Commands and executables
lammps_command              lmp_mpi
#mpi_command                 c:\\Program Files\\MPICH2\\bin\\mpiexec -localonly 4

# Build load information from crystal_space_group results
buildcombos                 elasticparent load_file parent
parent_record               calculation_elastic_constants_static
parent_load_key             system-info
parent_strainrange          1e-7

# Build defect records
buildcombos                 defect dislocation_file
defect_record               dislocation_monopole
defect_id                   A2--W--bcc--a-2-111--90-edge--{112}

# System manipulations
a_uvw                      
b_uvw                      
c_uvw                 
atomshift                   
sizemults                   0 1 -57 57 -15 15

# Units that input/output values are in
length_unit                 
pressure_unit               
energy_unit                 
force_unit                  

# Run parameters
annealtemperature           10
annealsteps                 100
randomseed
energytolerance             
forcetolerance              
maxiterations               
maxevaluations              
maxatommotion               
duplicatecutoff             
boundarywidth               
onlyuselinear               
"""

with open('input_script.in', 'w') as f:
    f.write(input_script)

### Parse input script

In [42]:
with open('input_script.in') as f:
    input_dict = iprPy.input.parse(f, singularkeys=calculation.singularkeys)

### Add parent_load_file list using unique crystals

In [43]:
input_dict['parent_load_file'] = []
for key in unique_crystal_keys:
    input_dict['parent_load_file'].append(key + '.json')

In [44]:
input_dict

{'lammps_command': 'lmp_mpi',
 'buildcombos': ['elasticparent load_file parent', 'defect dislocation_file'],
 'parent_record': 'calculation_elastic_constants_static',
 'parent_load_key': 'system-info',
 'parent_strainrange': '1e-7',
 'defect_record': 'dislocation_monopole',
 'defect_id': 'A2--W--bcc--a-2-111--90-edge--{112}',
 'sizemults': '0 1 -57 57 -15 15',
 'annealtemperature': '10',
 'annealsteps': '100',
 'parent_load_file': ['5028ecce-272a-4a11-a3e4-10f97993fb8e.json',
  '3fe58722-8e95-4489-860e-e8ac58261306.json',
  'a11f2008-6ab6-4f9b-b434-2d1dcfa935f7.json',
  '7ae00795-a00a-49f7-9511-a2dd795a3f77.json']}

### Prepare calculations

In [45]:
database.prepare(run_directory, calculation, **input_dict)

In [46]:
database.check_records(calculation.record_style)

In database style local at C:\Users\lmh1\Documents\calculations\ipr\demo :
- 8 of style calculation_dislocation_periodic_array
 - 0 are complete
 - 8 still to run
 - 0 issued errors


### Run calculations

In [ ]:
database.runner(run_directory)

Runner started with pid 14408


In [ ]:
results_df = database.get_records_df(style=calculation.record_style)
error_df = results_df[results_df.status=='error']
print(len(error_df), 'calculations issued errors:')
errors = []
for error in error_df.error:
    lines = error.splitlines()
    err = ''
    for i in range(len(lines)-1, -1, -1):
        if 'Error:' in lines[i]:
            err = '\n'.join(lines[i:-1])
            break
        if i == 0:
            err = error
    errors.append(err)
for error in np.unique(errors):
    print(error)